In [1]:
# dataframe management
import pandas as pd             

# numerical computation
import numpy as np

# visualization library
import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

#list operations
import collections

#import pyspark
#from datetime import datetime
#from pyspark.sql.types import IntegerType, FloatType
#from pyspark.sql.functions import udf,lit

#from pyspark.sql import SQLContext
#from pyspark.sql.functions import mean

#sc = pyspark.SparkContext.getOrCreate()
#sqlContext = SQLContext(sc)

# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")

from bokeh.layouts import gridplot
from bokeh.plotting import figure

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook()

pd.set_option('display.max_columns', None)


Loading BokehJS ...

In [2]:
sales_string_date = pd.read_csv('train.csv')

#dropping tuples where stores are closed
sales = sales_string_date[sales_string_date['IsOpen'] == 1]

#Imputation of missing values for Event
sales=sales.replace(np.nan,'NotSpecified', regex=True)

sales.head(5)

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,Events,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,Max_Sea_Level_PressurehPa,Max_TemperatureC,Max_VisibilityKm,Max_Wind_SpeedKm_h,Mean_Dew_PointC,Mean_Humidity,Mean_Sea_Level_PressurehPa,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees
0,1000,01/03/2016,0,1,0,Hyper Market,General,326,7,495,5676,9643,17130,2770,8,Rain-Snow,1,NotSpecified,100,1032,2,19,21,-1,82,1030,1,11,16,-2,70,1029,1,6,0.0,23
1,1000,02/03/2016,0,1,0,Hyper Market,General,326,7,608,8111,9643,17130,2770,8,Snow,0,NotSpecified,87,1030,5,23,16,-1,73,1027,3,13,10,-2,58,1025,1,10,0.0,56
2,1000,04/03/2016,0,1,0,Hyper Market,General,326,7,665,8300,9643,17130,2770,8,Rain,0,NotSpecified,81,1026,4,31,23,-1,71,1024,3,11,10,-3,55,1023,2,8,0.0,22
3,1000,05/03/2016,0,1,0,Hyper Market,General,326,7,630,7154,9643,17130,2770,6,NotSpecified,-3,NotSpecified,80,1027,8,31,19,-4,56,1024,3,15,10,-6,25,1022,-1,10,0.0,108
5,1000,07/03/2016,0,1,1,Hyper Market,General,326,7,763,10110,9643,17130,2770,NotSpecified,NotSpecified,-2,NotSpecified,93,1024,11,31,24,-3,65,1020,4,22,11,-4,26,1016,-2,16,0.0,68


In [3]:
sales_string_date.describe()

,StoreID,IsHoliday,IsOpen,HasPromotions,NearestCompetitor,Region,NumberOfCustomers,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,Max_Sea_Level_PressurehPa,Max_TemperatureC,Max_VisibilityKm,Max_Wind_SpeedKm_h,Mean_Dew_PointC,Mean_Humidity,Mean_Sea_Level_PressurehPa,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees
count,523021.000000,523021.000000,523021.000000,523021.000000,523021.00000,523021.000000,523021.000000,523021.000000,523021.000000,523021.000000,523021.000000,481840.000000,523021.000000,113074.000000,523021.000000,523021.000000,523021.000000,511683.000000,523021.000000,523021.000000,523021.000000,523021.000000,523021.000000,511683.000000,523021.000000,523021.000000,523021.000000,523021.000000,523021.000000,511683.000000,523021.000000,523021.000000
mean,1373.953914,0.029718,0.829714,0.383292,8002.71122,5.713151,259.325073,4057.377536,12381.265718,14050.547848,3861.519300,5.568627,8.512014,47.980782,93.523316,1018.133184,14.424012,23.560275,22.223437,6.229677,74.862845,1015.002422,10.273205,12.151025,11.724961,3.737584,51.375624,1012.078435,6.202315,7.223408,0.808278,170.458096
std,216.395820,0.169808,0.375884,0.486189,11537.70753,3.357190,185.857102,2729.630381,9140.692427,2755.425404,2816.719484,1.715277,5.988302,12.771727,7.617389,7.985673,8.196295,9.169704,8.725281,5.881529,12.939739,8.261523,7.041167,4.978385,5.791031,5.909098,19.284023,8.699236,6.312993,4.899575,2.631428,101.728193
min,1000.000000,0.000000,0.000000,0.000000,47.00000,0.000000,0.000000,0.000000,344.000000,9893.000000,816.000000,0.000000,-14.000000,21.000000,44.000000,976.000000,-11.000000,0.000000,3.000000,-15.000000,30.000000,974.000000,-13.000000,0.000000,2.000000,-18.000000,10.000000,970.000000,-15.000000,0.000000,0.000000,-1.000000
25%,1187.000000,0.000000,1.000000,0.000000,1057.00000,3.000000,166.000000,2573.000000,7215.000000,11849.000000,1293.000000,5.000000,4.000000,39.000000,90.000000,1013.000000,8.000000,11.000000,14.000000,2.000000,66.000000,1010.000000,4.000000,10.000000,8.000000,-1.000000,36.000000,1007.000000,1.000000,3.000000,0.000000,75.000000
50%,1373.000000,0.000000,1.000000,0.000000,3321.00000,6.000000,251.000000,4020.000000,9643.000000,15017.000000,2770.000000,6.000000,9.000000,47.000000,94.000000,1018.000000,15.000000,31.000000,21.000000,7.000000,76.000000,1015.000000,11.000000,11.000000,11.000000,4.000000,51.000000,1013.000000,7.000000,8.000000,0.000000,191.000000
75%,1561.000000,0.000000,1.000000,1.000000,9673.00000,9.000000,345.000000,5538.000000,15566.000000,15931.000000,5727.000000,7.000000,13.000000,55.000000,100.000000,1023.000000,21.000000,31.000000,26.000000,11.000000,85.000000,1020.000000,16.000000,14.000000,14.000000,8.000000,66.000000,1018.000000,11.000000,10.000000,0.250000,250.000000
max,1748.000000,1.000000,1.000000,1.000000,85070.00000,10.000000,2206.000000,26641.000000,32221.000000,23931.000000,8146.000000,8.000000,23.000000,106.000000,100.000000,1043.000000,37.000000,31.000000,101.000000,20.000000,100.000000,1040.000000,30.000000,31.000000,53.000000,18.000000,100.000000,1038.000000,24.000000,31.000000,58.930000,360.000000


In [4]:
stores = [store for store in sales["StoreID"]]
stores.sort()
stores = list(set(stores))
    

In [5]:
len(stores) #How many distinct stores we have. 

749

In [6]:
#Info on sales when no Events are reported

no_event_sales = sales[sales['Events'] == 'NotSpecified']
no_event_sales = no_event_sales[['StoreID', 'NumberOfSales']]
    
no_event_sales = no_event_sales.pivot_table(index='StoreID', values='NumberOfSales', aggfunc={np.median})

names = no_event_sales.columns.tolist()
names[names.index('median')] = 'NotSpecified'
no_event_sales.columns = names

no_event_sales[:10]

,NotSpecified
StoreID,
1000,7388.0
1001,2895.5
1002,5048.5
1003,5150.0
1004,3901.5
1005,4716.0
1006,6483.0
1007,3852.0
1008,5547.0


In [7]:
#Ordering events by frequency
count = collections.Counter(sales['Events'])
event_list = sorted(sales['Events'], key=count.get, reverse=True)

event_set = set(event_list)
temp = {"NotSpecified"}
event_set = event_set.difference(temp)
all_event_sales = no_event_sales

for event in event_set:
    event_sales = sales[sales['Events'] == event]
    event_sales = event_sales[['StoreID', 'NumberOfSales']]
    event_sales = event_sales.pivot_table(index='StoreID', values='NumberOfSales', aggfunc=np.median)
    
    all_event_sales = pd.concat([all_event_sales, event_sales], axis = 1)
    
    names = all_event_sales.columns.tolist()
    names[names.index('NumberOfSales')] = event
    all_event_sales.columns = names

all_event_sales[:10]


,NotSpecified,Fog-Snow,Rain-Snow-Hail-Thunderstorm,Rain-Snow,Rain-Hail,Snow,Fog-Rain-Snow,Fog,Fog-Rain-Snow-Hail,Thunderstorm,Rain,Fog-Rain-Thunderstorm,Fog-Snow-Hail,Fog-Thunderstorm,Fog-Rain,Rain-Snow-Hail,Fog-Rain-Hail,Rain-Hail-Thunderstorm,Rain-Snow-Thunderstorm,Fog-Rain-Hail-Thunderstorm,Snow-Hail,Rain-Thunderstorm
StoreID,,,,,,,,,,,,,,,,,,,,,,
1000,7388.0,NaN,NaN,6867.5,6628.0,7142.0,7884.0,7869.0,8907.0,NaN,7207.0,6892.5,NaN,NaN,7476.0,7562.0,NaN,NaN,NaN,6092.0,NaN,6659.5
1001,2895.5,2680.0,NaN,3705.5,NaN,2468.0,1877.0,2544.0,NaN,NaN,2898.0,2633.5,NaN,2744.0,2751.0,NaN,NaN,NaN,NaN,NaN,2503.5,3101.0
1002,5048.5,4324.0,NaN,4431.5,6500.0,4620.5,4844.0,4973.0,NaN,4671.0,4756.0,4241.0,NaN,NaN,4884.0,NaN,NaN,NaN,NaN,3613.0,NaN,4811.0
1003,5150.0,NaN,NaN,5528.0,4867.0,4390.0,5413.5,5375.0,6988.0,NaN,5313.5,4944.0,NaN,NaN,5206.0,5099.0,NaN,NaN,NaN,4633.0,NaN,4608.0
1004,3901.5,3619.0,NaN,4062.5,NaN,3753.0,3740.0,4001.0,NaN,3386.0,4073.0,3797.0,NaN,NaN,3921.5,NaN,NaN,NaN,NaN,NaN,NaN,3713.5
1005,4716.0,4524.0,5311.0,4797.0,4942.0,3877.0,4961.0,4678.5,5902.0,4478.0,4705.0,4099.0,3744.0,NaN,4584.5,4104.5,2643.0,NaN,NaN,NaN,NaN,4463.5
1006,6483.0,4460.0,NaN,6637.0,5689.0,6172.0,4577.0,6365.5,NaN,7260.5,6347.5,5650.0,NaN,NaN,6350.0,NaN,NaN,NaN,8974.0,NaN,NaN,6227.5
1007,3852.0,2580.0,NaN,3959.0,4208.0,3857.0,3372.0,3881.5,NaN,4584.0,3861.0,3462.0,NaN,NaN,3986.0,NaN,NaN,NaN,5338.0,NaN,NaN,4320.5
1008,5547.0,4811.5,7132.0,5454.0,5635.5,5678.0,6756.0,5077.5,4952.0,4710.0,5507.0,4265.0,4610.0,NaN,5183.5,4945.5,5473.0,NaN,NaN,NaN,NaN,4825.0


In [8]:
attribute_list = all_event_sales.columns.tolist()
attribute_list.remove('NotSpecified')
increments = list()
variances = []

for attribute in attribute_list:
    for index, row in all_event_sales.iterrows():
        no_event_median = row['NotSpecified']
        event_median = row[attribute]
        
        #print(no_event_median, event_median)
        
        if event_median == event_median: #check if not Nan
            increment = -(no_event_median - event_median)/no_event_median
            increments.append(increment)
        
    median_of_medians = np.median(increments)
    variances.append((attribute, median_of_medians))
    
    
variances[:100]
    
#sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 1})
#sns.boxplot(x="Region", y="Min_VisibilitykM", data=sales)

[('Fog-Snow', -0.07967083827723145),
 ('Rain-Snow-Hail-Thunderstorm', -0.05154777927321669),
 ('Rain-Snow', -9.289935154844546e-05),
 ('Rain-Hail', -0.001578778962218639),
 ('Snow', -0.011711711711711712),
 ('Fog-Rain-Snow', -0.01699078812691914),
 ('Fog', -0.013038800276976993),
 ('Fog-Rain-Snow-Hail', -0.006690777576853526),
 ('Thunderstorm', -0.015197086893064985),
 ('Rain', -0.011010136315973435),
 ('Fog-Rain-Thunderstorm', -0.021301089752798975),
 ('Fog-Snow-Hail', -0.02458794920291813),
 ('Fog-Thunderstorm', -0.02419362354667295),
 ('Fog-Rain', -0.019240065744948274),
 ('Rain-Snow-Hail', -0.01808336173654705),
 ('Fog-Rain-Hail', -0.020095927622993198),
 ('Rain-Hail-Thunderstorm', -0.020496504641438172),
 ('Rain-Snow-Thunderstorm', -0.01911251454213063),
 ('Fog-Rain-Hail-Thunderstorm', -0.02134866866236017),
 ('Snow-Hail', -0.020859440267335),
 ('Rain-Thunderstorm', -0.023026315789473683)]